**Designing a Relational Database Schema for a Library Information System**

---

### 📄 Specification and Objective

The goal is to design a relational database schema for an **Institute Library Information System (LIS)** based on a natural language specification document. The core objective is to develop a schema that:

* Is derived using **Entity-Relationship (ER) modeling**
* Is transformed into relational schemas
* Satisfies **BCNF**, or at minimum, **3NF**
* Supports efficient **querying and constraint enforcement**

---

### 🔍 Key Functional Requirements (Extracted from Specification)

* Library manages a large number of **books** and **members**
* Books are **issued** and **returned** by members
* Each **book** has: title, authors (first author recorded), publisher, year, ISBN (unique per title), accession number (unique per physical copy)
* Multiple copies of the same book exist
* **Members** can be:

  * Undergraduate students
  * Postgraduate students
  * Research scholars
  * Faculty members
* Members have:

  * Roll number (for students)
  * ID (for faculty)
  * Membership number (unique for all)
* Member **quotas** specify:

  * Number of books allowed
  * Time duration per issued book
* Constraints:

  * A book can only be issued if it is not already issued
  * A member cannot issue multiple copies of the same book
  * No books can be issued if member has overdue books
  * Quota rules must be respected

---

### 🖊️ ER Modeling Breakdown

#### **Entities and Attributes**

* **Book**: ISBN (unique per title), title, publisher, year, first author
* **BookCopy**: accession number (primary key), ISBN (foreign key)
* **Member**: member number (primary key), member type (UG, PG, RS, Faculty), member class (student/faculty), roll number / ID (nullable)
* **Student**: roll number (primary key), name, department, degree, date of joining, date of birth, mobile (nullable)
* **Faculty**: ID (primary key), name, department, designation, mobile (nullable), date of joining, date of birth
* **Quota**: member type (primary key), max\_books, max\_duration
* **Staff** (speculative): staff ID, name, gender, mobile, date of joining

#### **Relationships**

* **BookIssue**: accession number, member number, date of issue

  * Many-to-One from books to members
  * Relationship attribute: date of issue (DOI)

---

### 🔧 Initial Relational Schema

#### Core Relations

* **Books**(ISBN, title, author\_fname, author\_lname, publisher, year)
* **BookCopies**(accession\_no PRIMARY KEY, ISBN FOREIGN KEY)
* **Members**(member\_no PRIMARY KEY, member\_type, member\_class, roll\_no, faculty\_id)
* **Students**(roll\_no PRIMARY KEY, member\_no, name, degree, department, doj, dob, mobile)
* **Faculty**(faculty\_id PRIMARY KEY, member\_no, name, department, designation, doj, dob, mobile)
* **Quota**(member\_type PRIMARY KEY, max\_books, max\_duration)
* **BookIssue**(accession\_no, member\_no, DOI, PRIMARY KEY (accession\_no, member\_no))
* **Staff**(staff\_id PRIMARY KEY, name, gender, mobile, doj)

---

### 🔹 Normalization to BCNF

#### 🔜 Books & BookCopies

* Books: ISBN ➞ title, author, publisher, year ✅ BCNF
* BookCopies: accession\_no ➞ ISBN ✅ BCNF

#### 🔜 BookIssue

* FD: accession\_no, member\_no ➞ DOI ✅ BCNF

#### 🔜 Quota

* FD: member\_type ➞ max\_books, max\_duration ✅ BCNF

#### 🔜 Members

* Requires redesign for usability: Add member\_class and reference IDs (nullable)

#### 🔜 Students & Faculty

* Roll\_no and faculty\_id uniquely identify students and faculty respectively
* But querying name of the member who issued a book requires knowing class ➔ Add `member_class` and ID/roll as nullable fields to `Members`

---

### 📊 Refinement for Query Efficiency

Problem: Querying "Who issued a book with accession X?" is complex due to split tables.

**Refined Design**:

* Add `member_class` and either `roll_no` or `faculty_id` in `Members`
* Add NULLABLE fields to allow union-style querying

**Updated Members Schema**:

```sql
Members(member_no PRIMARY KEY, member_type, member_class, roll_no NULLABLE, faculty_id NULLABLE)
```

**Sample Query Logic**:

```sql
SELECT name FROM Students WHERE roll_no =
  (SELECT roll_no FROM Members WHERE member_no = X AND member_class = 'student')
UNION
SELECT name FROM Faculty WHERE faculty_id =
  (SELECT faculty_id FROM Members WHERE member_no = X AND member_class = 'faculty')
```

---

### 🚀 Final Schema Snapshot

| Relation   | Primary Key               | Foreign Key Reference(s)                            |
| ---------- | ------------------------- | --------------------------------------------------- |
| Books      | ISBN                      | -                                                   |
| BookCopies | accession\_no             | ISBN → Books                                        |
| Members    | member\_no                | -                                                   |
| Students   | roll\_no                  | member\_no → Members                                |
| Faculty    | faculty\_id               | member\_no → Members                                |
| Quota      | member\_type              | -                                                   |
| BookIssue  | accession\_no, member\_no | accession\_no → BookCopies,<br>member\_no → Members |
| Staff      | staff\_id                 | -                                                   |

---

### 🔮 Learning Outcome

You now understand how to:

* Extract entities and relationships from a specification document
* Model in ER form and transform to relational schema
* Normalize schema to BCNF/3NF
* Refine schema for real-world query support
* Balance theoretical design with practical efficiency for LIS

This methodology can be generalized to **any real-world application domain** requiring robust relational schema design.
